# Модели первого уровня

Будем использовать lightFM с подобранными параметрами из предыдущего дз, потом добавим в нее популярную модель для работы с холодными юзерами

## Импорты

In [1]:
import warnings
warnings.simplefilter('ignore')

import dill
import numpy as np
import pandas as pd

import rectools
from rectools.models import PopularModel
from lightfm import LightFM
from lightfm.data import Dataset
from pathlib import Path
from rectools.metrics import calc_metrics, NDCG, MAP, Precision, Recall, MeanInvUserFreq
from rectools import Columns
from sklearn.model_selection import train_test_split
from typing import Any, Dict
from tqdm.auto import tqdm
from notebooks.tools  import generate_lightfm_recs_mapper

## Подготовка данных (из 4 дз + дополнения из 6 практики)

In [2]:
DATA_PATH = Path("data_original")

In [3]:
%%time
users = pd.read_csv(DATA_PATH / 'users.csv')
items = pd.read_csv(DATA_PATH / 'items.csv')
interactions = pd.read_csv(DATA_PATH / 'interactions.csv')

CPU times: total: 1.33 s
Wall time: 2.02 s


In [4]:
# Меняем названия колонок для использования rectools
interactions.rename(
    columns={
        'last_watch_dt': Columns.Datetime,
        "watched_pct": Columns.Weight,
    },
    inplace=True,
)

interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime], format='%Y-%m-%d')

# Заполняем пропуски
interactions_default_values: Dict[str, Any] = {
   Columns.Datetime: interactions[Columns.Datetime].median(),
    Columns.Weight: 0.,
    'total_dur': 0,
}
interactions.fillna(interactions_default_values, inplace=True)

max_date = interactions[Columns.Datetime].max()
#вставим сюда только тот промежуток, на котором не учится ранкер
ranker_days_count = 30
train = interactions[(interactions[Columns.Datetime] < max_date - pd.Timedelta(days=ranker_days_count))]


In [5]:
train

,user_id,item_id,datetime,total_dur,weight
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0
...,...,...,...,...,...
5476242,268216,3071,2021-04-21,5752,98.0
5476243,497899,9629,2021-05-29,45,1.0
5476245,786732,4880,2021-05-12,753,0.0
5476247,546862,9673,2021-04-13,2308,49.0


In [6]:
# Обучать ранжирование будем на последнем месяце (30 дней) не считая отложенной недели
# Лучше зафиксировать временной диапазон если на проде планируется переобучать модель
ranker_days_count = 30

ranker_data = interactions[
    (interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=ranker_days_count))
]

# В дальнейшем ranker_data разбиваем по юзерам
#  на train val test для обучения, валидации и тестирования ранкера
train_size = 0.7
val_size = 0.15
test_size = 0.15

# В train_test_split очень удобно можно сохранить исходное распределение по нужным факторам,
#  задав параметр stratify. Правда мы на это пока забьем)

train_val_users, test_users = train_test_split(
    ranker_data['user_id'].unique(), random_state=42, test_size=test_size
)

train_users, val_users = train_test_split(
    train_val_users, random_state=42, test_size=val_size / (train_size + val_size)  # 15% от общего размера
)

## Обучаем модель первого уровня

### Обучаем LightFM

In [7]:
lightfm_dataset = Dataset()
lightfm_user_ids = train['user_id'].unique()
lightfm_item_ids = train['item_id'].unique()
lightfm_dataset.fit(lightfm_user_ids, lightfm_item_ids)

In [8]:
# Тогда матрицу интеракций и весов можно получить следующим образом:
interactions_matrix, weights_matrix = lightfm_dataset.build_interactions(
    zip(*train[['user_id', 'item_id', Columns.Weight]].values.T)
)
weights_matrix = weights_matrix.tocsr()

In [9]:
# Обучаем модель
# Не используем доступные фичи юзеров и айтемов, оставим это на этап реранжирования

lfm_model = LightFM(
    no_components=48,
    learning_rate=0.0099,
    loss='warp',
    max_sampled=5,
    random_state=42,
)

num_epochs = 20

for _ in tqdm(range(num_epochs)):
    lfm_model.fit_partial(weights_matrix)

  0%|          | 0/20 [00:00<?, ?it/s]

In [10]:
# save model
with open(f'lightfm_model_.dill', 'wb') as f:
    dill.dump(lfm_model, f)

### Кандидаты LightFm

In [11]:
# Маппинги обычных айдишников во внутренние индексы lightfm для юзеров и айтемов
lightfm_mapping = lightfm_dataset.mapping()
lightfm_mapping = {
    'user_id_to_iid': lightfm_mapping[0],  # iid - internal lfm id
    'item_id_to_iid': lightfm_mapping[2],
}
# Маппинги внутренние индексов lightfm в обычные айдишники для юзеров и айтемов
lightfm_mapping['user_iid_to_id'] = {v: k for k, v in lightfm_mapping['user_id_to_iid'].items()}
lightfm_mapping['item_iid_to_id'] = {v: k for k, v in lightfm_mapping['item_id_to_iid'].items()}

In [12]:
# Топ 50 кандидатов
top_N = 50

# Внутренние индексы юзеров и айтемов lightfm (индексы матрицы user-item)
user_lfm_index = np.array(list(lightfm_mapping['user_id_to_iid'].values()))
item_lfm_index = np.array(list(lightfm_mapping['item_id_to_iid'].values()))

# Делаем предикт, ограничиваем его сверху top_N
# и сразу переводим в настоящие айдишники
mapper = generate_lightfm_recs_mapper(
    model=lfm_model,
    N=top_N,
    item_iids=item_lfm_index,
    user_id_to_iid=lightfm_mapping['user_id_to_iid'],
    item_iid_to_id=lightfm_mapping['item_iid_to_id'],
    known_item_ids=dict(),  # тут можно добавить уже просмотренный контент для его исключения
    num_threads=4,
)

In [13]:
# Генерируем предсказания и получаем скоры и ранги lightfm

candidates = pd.DataFrame({'user_id': lightfm_user_ids})
candidates['item_id'], candidates['lfm_score'] = zip(*candidates['user_id'].map(mapper))
candidates = candidates.explode(['item_id', 'lfm_score'], ignore_index=True)
candidates['lfm_rank'] = candidates.groupby('user_id').cumcount() + 1

candidates= pd.read_csv('candidates.csv')

### Обучаем популярную модель как в одной из прошлых дз

Так как мы дополняем лайт фм, то будем использовать популярное для холодных юзеров

In [14]:
popular_dataset = rectools.dataset.Dataset.construct(train)
popular_model = PopularModel()
popular_model.fit(popular_dataset)

In [15]:
k = items[Columns.Item].nunique()
popular_recos = popular_model.recommend(
    popular_dataset.user_id_map.external_ids[:1], popular_dataset, k, True
)
popular_recos.rename({"rank": "popular_rank", "score": "popular_score"}, axis=1, inplace=True)
popular_recos.drop(Columns.User, axis=1, inplace=True)
popular_recos

,item_id,popular_score,popular_rank
0,10440,141889.0,1
1,15297,137128.0,2
2,13865,93403.0,3
3,4151,69641.0,4
4,2657,55146.0,5
...,...,...,...
14936,15235,1.0,14937
14937,87,1.0,14938
14938,7457,1.0,14939
14939,6270,1.0,14940


## Результат моделей первого уровня

In [16]:
recos_result = candidates.merge(popular_recos, how="left", on=[Columns.Item])


In [17]:
# Считаем метрики
def calc_metrics_(candidates_df, rank_col: str) -> Dict[str, float]:
    metrics = {
        'ndcg@10': NDCG(k = 10),
        'map@10': MAP(k = 10),
        'Precision@10': Precision(k = 10),
        'recall@10': Recall(k = 10),
        'novelty@10': MeanInvUserFreq(k = 10),
    }
    return calc_metrics(
        metrics=metrics,
        reco=(
            candidates_df
            .rename(columns={rank_col: Columns.Rank})
            [[Columns.User, Columns.Item, Columns.Rank]]
            [candidates_df[Columns.User].isin(test_users)]
        ),
        interactions=(
            ranker_data
            [[Columns.User, Columns.Item, Columns.Datetime, Columns.Weight]]
            [ranker_data[Columns.User].isin(test_users)]
        ),
        prev_interactions=(
            train
            [[Columns.User, Columns.Item, Columns.Datetime, Columns.Weight]]
            [train[Columns.User].isin(test_users)]
        ),
        catalog=items['item_id'].unique()
    )

models_metrics: Dict[str, Dict[str, float]] = dict()
models_metrics['lfm'] = calc_metrics_(candidates, 'lfm_rank')
models_metrics['lfm']

{'Precision@10': 0.02438567679155219,
 'recall@10': 0.0809703106614082,
 'ndcg@10': 0.02903718748141727,
 'map@10': 0.03701412518139648,
 'novelty@10': 3.1777199914123955}

In [18]:
models_metrics['popular'] = calc_metrics_(recos_result, "popular_rank")

In [19]:
pd.DataFrame(models_metrics)[["lfm", "popular"]]

,lfm,popular
Precision@10,0.024386,0.019285
recall@10,0.080970,0.065335
ndcg@10,0.029037,0.023902
map@10,0.037014,0.031254
novelty@10,3.177720,3.025827


In [20]:
# Делаем чекпоинт - сохраняем кандидатов
recos_result.to_csv('candidates_ranker.csv', index=False)


In [21]:
recos_result

,user_id,item_id,lfm_score,lfm_rank,popular_score,popular_rank
0,176549,9728,4.125431,1,NaN,NaN
1,176549,12173,4.028789,2,10182.0,28.0
2,176549,13865,4.011769,3,93403.0,3.0
3,176549,7626,4.011071,4,NaN,NaN
4,176549,13018,3.934987,5,NaN,NaN
...,...,...,...,...,...,...
36043745,805174,12463,1.480902,46,7447.0,41.0
36043746,805174,7829,1.463453,47,6028.0,58.0
36043747,805174,6402,1.461368,48,7761.0,40.0
36043748,805174,12501,1.45889,49,7387.0,45.0
